# EEG - Flow

## 2. Select bad channels and annotate bad segments

The bad channels and the bad segments must be annotated and excluded from subsequent analysis. 

Last edit: 07.05.2023 01:43
@anguyen

In [13]:
import os

from mne import set_log_level as set_log_level_mne
from mne.io import write_info
from mne.viz import set_browser_backend

from eeg_flow import set_log_level
from eeg_flow.tasks import annotate_bad_channels_and_segments

In [14]:
set_log_level_mne("WARNING")
set_log_level("INFO")

%matplotlib qt
set_browser_backend("qt")

'qt'

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [15]:
PARTICIPANT: str = "P02"  # e.g. "P02"
GROUP: str = "G2"  # e.g. "G2"
TASK: str = "oddball"  # "oddball" or "UT"
RUN: int = 1  # 1 or 2

This step is divided in:
- Visualization of gel-bridges and rejection of the dataset if required
- Interpolation of gel-bridges
- Annotation of bad channels with the PREP pipeline
- Visualization of the dataset to confirm the bad channels and to annotate bad segments

The created derivatives are:
- Info containing the bad channels (`_info.fif`)
- Oddball task annotation + bad segment annotation (`_oddball_with_bads_annot.fif`)
- Plot of the gel-bridges (`_bridges.svg`)
- Raw with interpolated gel-bridged channels (`_raw.fif`)

More information about gel bridgs in this [MNE tutorial](https://mne.tools/stable/auto_examples/preprocessing/eeg_bridging.html).

In [16]:
annotate_bad_channels_and_segments(PARTICIPANT, GROUP, TASK, RUN)

[IN] Do you want to continue with this dataset? [Y/n]  y


2023-05-24 14:49:04,887 - qdarkstyle - INFO - QSS file successfully loaded.
2023-05-24 14:49:04,888 - qdarkstyle - INFO - Found version patches to be applied.
2023-05-24 14:49:04,888 - qdarkstyle - INFO - Found application patches to be applied.


RuntimeError: Info file sub-P02-G2_task-oddball_run-1_step2_info.fif does already exist.